# Обработка датасета

## Библиотеки и скрипты

In [18]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Библиотека для предобработки датасетов
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_preprocessing import DataPreprocessor
dp = DataPreprocessor()

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

In [19]:
# Получение ClientID
def get_id(x):
    id = re.search('YA:.*', x).group(0)
    return re.sub('YA:', '', id).strip()

## Обзор данных

In [20]:
# Загрузка датасета
df = pd.read_csv('../data/LEAD.csv', encoding='cp1251', sep=';')

In [21]:
df.head(5)

,ID,Статус,Название лида,Обращение,Имя,Отчество,Фамилия,Дата рождения,Дата создания,Источник,...,Причина отказа лида (список),Партнерство,Подписан на рассылку,UF_CRM_TEXTAREA,UF_CRM_TRANID,UF_CRM_FORMNAME,UF_CRM_COOKIES,UF_CRM_VASHESOOBSCHE,UF_CRM_INPUT,Отправка списка компонентов
0,10349,Не обработан,Лид #10349,NaN,NaN,NaN,NaN,NaN,28.04.2021 11:21,Веб-сайт HubEx.ru,...,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10347,Обработан,ТОО TechQyzmet KZ,NaN,Алимжан Даржанов,NaN,NaN,NaN,28.04.2021 6:43,Веб-сайт HubEx.ru,...,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10345,Обработан,ИП Кирьянов,NaN,Евгений Кирьянов,NaN,NaN,NaN,27.04.2021 23:24,Веб-сайт HubEx.ru,...,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10337,Обработан,Сады Придонья,NaN,Елена Евгеньевна Слетова,NaN,NaN,NaN,27.04.2021 15:31,Веб-сайт HubEx.ru,...,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10335,Ожидает квалификации,Райт левел,Муж,Кирилл Герасимов,NaN,NaN,NaN,27.04.2021 11:44,Веб-сайт HubEx.ru,...,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.shape

(1468, 91)

## Обработка данных

In [23]:
df.columns

Index(['ID', 'Статус', 'Название лида', 'Обращение', 'Имя', 'Отчество',
       'Фамилия', 'Дата рождения', 'Дата создания', 'Источник',
       'Рабочий телефон', 'Мобильный телефон', 'Номер факса',
       'Домашний телефон', 'Номер пейджера', 'Телефон для рассылок',
       'Другой телефон', 'Корпоративный сайт', 'Личная страница',
       'Страница Facebook', 'Страница ВКонтакте', 'Страница LiveJournal',
       'Микроблог Twitter', 'Другой сайт', 'Рабочий e-mail', 'Частный e-mail',
       'E-mail для рассылок', 'Другой e-mail', 'Контакт Facebook',
       'Контакт Telegram', 'Контакт ВКонтакте', 'Контакт Skype',
       'Контакт Viber', 'Комментарии Instagram', 'Контакт Битрикс24.Network',
       'Онлайн-чат', 'Контакт Открытая линия', 'Контакт ICQ',
       'Контакт MSN/Live!', 'Контакт Jabber', 'Другой контакт',
       'Ответственный', 'Дополнительно о статусе',
       'Дополнительно об источнике', 'Кем создан', 'Дата изменения',
       'Кем изменен', 'Название компании', 'Должность', 'А

In [24]:
# Удаление ненужных данных
columns = ['Дата рождения', 'Рабочий телефон', 'Мобильный телефон', 'Номер факса', 'Домашний телефон', 'Номер пейджера', 
           'Телефон для рассылок', 'Другой телефон', 'Личная страница', 'Страница Facebook', 'Страница ВКонтакте', 
           'Страница LiveJournal', 'Микроблог Twitter', 'Другой сайт', 'Частный e-mail', 'E-mail для рассылок', 
           'Другой e-mail', 'Контакт Facebook', 'Контакт Telegram', 'Контакт ВКонтакте', 'Контакт Skype', 'Контакт Viber', 
           'Комментарии Instagram', 'Контакт Битрикс24.Network', 'Онлайн-чат', 'Контакт Открытая линия', 'Контакт ICQ', 
           'Контакт MSN/Live!', 'Контакт Jabber', 'Другой контакт', 'Ответственный', 'Дополнительно о статусе', 'Кем создан', 
           'Дата изменения', 'Кем изменен', 'Улица, номер дома', 'Квартира, офис, комната, этаж', 'Район', 'Регион', 
           'Почтовый индекс', 'Страна', 'Комментарий', 'Создан CRM-формой', 'Клиент', 'Путь клиента', 'Компоненты', 
           'Причина отказа сделки', 'Причина отказа сделки (список)', 'Проект', 'Конфигурация 1С', 'Наличие 1С', 
           'Причина отказа лида (список)', 'Партнерство', 'Подписан на рассылку', 'UF_CRM_TEXTAREA', 'UF_CRM_TRANID', 
           'UF_CRM_FORMNAME', 'UF_CRM_COOKIES', 'UF_CRM_VASHESOOBSCHE', 'Отправка списка компонентов']
df = dp.drop_col(df, columns)
df.head(2)

,ID,Статус,Название лида,Обращение,Имя,Отчество,Фамилия,Дата создания,Источник,Корпоративный сайт,...,UTM Campaign,UTM Content,UTM Term,Классификатор отрасли,Ёмкость лицензии,Отрасль,Отрасль (список),Страница в интернете,Источник трафика,UF_CRM_INPUT
0,10349,Не обработан,Лид #10349,NaN,NaN,NaN,NaN,28.04.2021 11:21,Веб-сайт HubEx.ru,NaN,...,all_main_russia_poisk,sistema_upravlenie_sotrudnikami,NaN,NaN,10.0,"Управление недвижимостью, ЖКХ - Клининг",NaN,NaN,Контекстная реклама,NaN
1,10347,Обработан,ТОО TechQyzmet KZ,NaN,Алимжан Даржанов,NaN,NaN,28.04.2021 6:43,Веб-сайт HubEx.ru,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Поиск Яндекс,NaN


In [25]:
# Переименование столбцов
df.columns = dp.get_translite(df.columns, 'crm_')
df.columns

Index(['crm_id', 'crm_status', 'crm_nazvanie_lida', 'crm_obraschenie',
       'crm_imja', 'crm_otchestvo', 'crm_familija', 'crm_data_sozdanija',
       'crm_istochnik', 'crm_korporativnyj_sajt', 'crm_rabochij_e_mail',
       'crm_dopolnitelno_ob_istochnike', 'crm_nazvanie_kompanii',
       'crm_dolzhnost', 'crm_adres', 'crm_naselennyj_punkt', 'crm_summa',
       'crm_valjuta', 'crm_povtornyj_lid', 'crm_utm_source', 'crm_utm_medium',
       'crm_utm_campaign', 'crm_utm_content', 'crm_utm_term',
       'crm_klassifikator_otrasli', 'crm_emkost_litsenzii', 'crm_otrasl',
       'crm_otrasl_spisok', 'crm_stranitsa_v_internete',
       'crm_istochnik_trafika', 'crm_uf_crm_input'],
      dtype='object')

In [26]:
# Убираем 2020 год (там одни тесты)
# df = df[df['crm_data_sozdanija'].str.contains('.*2021.*')]

**Обработка пропусков**

In [27]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_id                          1468 non-null   int64  
 1   crm_status                      1468 non-null   object 
 2   crm_nazvanie_lida               1468 non-null   object 
 3   crm_obraschenie                 379 non-null    object 
 4   crm_imja                        1336 non-null   object 
 5   crm_otchestvo                   53 non-null     object 
 6   crm_familija                    194 non-null    object 
 7   crm_data_sozdanija              1468 non-null   object 
 8   crm_istochnik                   1460 non-null   object 
 9   crm_korporativnyj_sajt          151 non-null    object 
 10  crm_rabochij_e_mail             1299 non-null   object 
 11  crm_dopolnitelno_ob_istochnike  1278 non-null   object 
 12  crm_nazvanie_kompanii           10

crm_id                               0
crm_status                           0
crm_nazvanie_lida                    0
crm_obraschenie                   1089
crm_imja                           132
crm_otchestvo                     1415
crm_familija                      1274
crm_data_sozdanija                   0
crm_istochnik                        8
crm_korporativnyj_sajt            1317
crm_rabochij_e_mail                169
crm_dopolnitelno_ob_istochnike     190
crm_nazvanie_kompanii              403
crm_dolzhnost                     1082
crm_adres                         1379
crm_naselennyj_punkt              1390
crm_summa                            0
crm_valjuta                          0
crm_povtornyj_lid                    0
crm_utm_source                    1352
crm_utm_medium                    1353
crm_utm_campaign                  1352
crm_utm_content                   1356
crm_utm_term                      1449
crm_klassifikator_otrasli         1164
crm_emkost_litsenzii     

In [28]:
# Корректировка значений
df.loc[df['crm_obraschenie'].isnull(), 'crm_obraschenie'] = 'undefined'
df.loc[df['crm_imja'].isnull(), 'crm_imja'] = 'undefined'
df.loc[df['crm_otchestvo'].isnull(), 'crm_otchestvo'] = 'undefined'
df.loc[df['crm_familija'].isnull(), 'crm_familija'] = 'undefined'
df.loc[df['crm_istochnik'].isnull(), 'crm_istochnik'] = 'undefined'
df.loc[df['crm_korporativnyj_sajt'].isnull(), 'crm_korporativnyj_sajt'] = 'undefined'
df.loc[df['crm_rabochij_e_mail'].isnull(), 'crm_rabochij_e_mail'] = 'undefined'
df.loc[df['crm_dopolnitelno_ob_istochnike'].isnull(), 'crm_dopolnitelno_ob_istochnike'] = 'undefined' # Для регулярки
df.loc[df['crm_nazvanie_kompanii'].isnull(), 'crm_nazvanie_kompanii'] = 'undefined'
df.loc[df['crm_dolzhnost'].isnull(), 'crm_dolzhnost'] = 'undefined'
df.loc[df['crm_adres'].isnull(), 'crm_adres'] = 'undefined'
df.loc[df['crm_naselennyj_punkt'].isnull(), 'crm_naselennyj_punkt'] = 'undefined'
df.loc[df['crm_utm_source'].isnull(), 'crm_utm_source'] = 'undefined'
df.loc[df['crm_utm_medium'].isnull(), 'crm_utm_medium'] = 'undefined'
df.loc[df['crm_utm_campaign'].isnull(), 'crm_utm_campaign'] = 'undefined'
df.loc[df['crm_utm_content'].isnull(), 'crm_utm_content'] = 'undefined'
df.loc[df['crm_utm_term'].isnull(), 'crm_utm_term'] = 'undefined'
df.loc[df['crm_klassifikator_otrasli'].isnull(), 'crm_klassifikator_otrasli'] = 'undefined'
df.loc[df['crm_emkost_litsenzii'].isnull(), 'crm_emkost_litsenzii'] = df['crm_emkost_litsenzii'].median()
df.loc[df['crm_otrasl'].isnull(), 'crm_otrasl'] = 'undefined'
df.loc[df['crm_otrasl_spisok'].isnull(), 'crm_otrasl_spisok'] = 'undefined'
df.loc[df['crm_stranitsa_v_internete'].isnull(), 'crm_stranitsa_v_internete'] = 'undefined'
df.loc[df['crm_istochnik_trafika'].isnull(), 'crm_istochnik_trafika'] = 'undefined'

In [29]:
# Получаем ClientID
df['crm_client_id'] = df['crm_dopolnitelno_ob_istochnike'].apply(lambda x: get_id(x) if 'YA' in x else 'undefined', 1)
df.loc[(df['crm_client_id'].isnull()) | (df['crm_client_id'] == ''), 'crm_client_id'] = 'undefined'

In [30]:
df.head(5)

,crm_id,crm_status,crm_nazvanie_lida,crm_obraschenie,crm_imja,crm_otchestvo,crm_familija,crm_data_sozdanija,crm_istochnik,crm_korporativnyj_sajt,...,crm_utm_content,crm_utm_term,crm_klassifikator_otrasli,crm_emkost_litsenzii,crm_otrasl,crm_otrasl_spisok,crm_stranitsa_v_internete,crm_istochnik_trafika,crm_uf_crm_input,crm_client_id
0,10349,Не обработан,Лид #10349,undefined,undefined,undefined,undefined,28.04.2021 11:21,Веб-сайт HubEx.ru,undefined,...,sistema_upravlenie_sotrudnikami,undefined,undefined,10.0,"Управление недвижимостью, ЖКХ - Клининг",undefined,undefined,Контекстная реклама,NaN,1591642943190342147
1,10347,Обработан,ТОО TechQyzmet KZ,undefined,Алимжан Даржанов,undefined,undefined,28.04.2021 6:43,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,10.0,undefined,undefined,undefined,Поиск Яндекс,NaN,1618056694119409516
2,10345,Обработан,ИП Кирьянов,undefined,Евгений Кирьянов,undefined,undefined,27.04.2021 23:24,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,10.0,undefined,undefined,undefined,Сайт helpdeski,NaN,1619554758624285553
3,10337,Обработан,Сады Придонья,undefined,Елена Евгеньевна Слетова,undefined,undefined,27.04.2021 15:31,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,"5. Промышленность, нефтегаз, ТЭК\t/\tОбслужива...",10.0,"Промышленность, производство, ТЭК - Сервис обо...",undefined,pridonie.ru,Поиск Яндекс,NaN,1619526373198768982
4,10335,Ожидает квалификации,Райт левел,Муж,Кирилл Герасимов,undefined,undefined,27.04.2021 11:44,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,"7. Управление недвижимостью, ЖКХ\t/\tБЦ",15.0,Выездное обслуживание оборудования - Технологи...,undefined,undefined,Контекстная реклама,NaN,undefined


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_id                          1468 non-null   int64  
 1   crm_status                      1468 non-null   object 
 2   crm_nazvanie_lida               1468 non-null   object 
 3   crm_obraschenie                 1468 non-null   object 
 4   crm_imja                        1468 non-null   object 
 5   crm_otchestvo                   1468 non-null   object 
 6   crm_familija                    1468 non-null   object 
 7   crm_data_sozdanija              1468 non-null   object 
 8   crm_istochnik                   1468 non-null   object 
 9   crm_korporativnyj_sajt          1468 non-null   object 
 10  crm_rabochij_e_mail             1468 non-null   object 
 11  crm_dopolnitelno_ob_istochnike  1468 non-null   object 
 12  crm_nazvanie_kompanii           14

In [32]:
# Выводим имена столцов
dp.get_idx_col(df.columns)

0 crm_id
1 crm_status
2 crm_nazvanie_lida
3 crm_obraschenie
4 crm_imja
5 crm_otchestvo
6 crm_familija
7 crm_data_sozdanija
8 crm_istochnik
9 crm_korporativnyj_sajt
10 crm_rabochij_e_mail
11 crm_dopolnitelno_ob_istochnike
12 crm_nazvanie_kompanii
13 crm_dolzhnost
14 crm_adres
15 crm_naselennyj_punkt
16 crm_summa
17 crm_valjuta
18 crm_povtornyj_lid
19 crm_utm_source
20 crm_utm_medium
21 crm_utm_campaign
22 crm_utm_content
23 crm_utm_term
24 crm_klassifikator_otrasli
25 crm_emkost_litsenzii
26 crm_otrasl
27 crm_otrasl_spisok
28 crm_stranitsa_v_internete
29 crm_istochnik_trafika
30 crm_uf_crm_input
31 crm_client_id
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [33]:
# Новый порядок столбцов
new_order = [31, 0, 7, 1, 2, 3, 4, 5, 6, 13, 12, 8, 9, 10, 11, 14, 15, 28, 29, 25, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 30]
df = df[df.columns[new_order]]

In [34]:
df.columns

Index(['crm_client_id', 'crm_id', 'crm_data_sozdanija', 'crm_status',
       'crm_nazvanie_lida', 'crm_obraschenie', 'crm_imja', 'crm_otchestvo',
       'crm_familija', 'crm_dolzhnost', 'crm_nazvanie_kompanii',
       'crm_istochnik', 'crm_korporativnyj_sajt', 'crm_rabochij_e_mail',
       'crm_dopolnitelno_ob_istochnike', 'crm_adres', 'crm_naselennyj_punkt',
       'crm_stranitsa_v_internete', 'crm_istochnik_trafika',
       'crm_emkost_litsenzii', 'crm_summa', 'crm_valjuta', 'crm_povtornyj_lid',
       'crm_utm_source', 'crm_utm_medium', 'crm_utm_campaign',
       'crm_utm_content', 'crm_utm_term', 'crm_klassifikator_otrasli',
       'crm_otrasl', 'crm_otrasl_spisok', 'crm_uf_crm_input'],
      dtype='object')

In [35]:
# Приведение типов
df = dp.astype_col(df, ['crm_emkost_litsenzii'], coltype='uint8')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_client_id                   1468 non-null   object 
 1   crm_id                          1468 non-null   int64  
 2   crm_data_sozdanija              1468 non-null   object 
 3   crm_status                      1468 non-null   object 
 4   crm_nazvanie_lida               1468 non-null   object 
 5   crm_obraschenie                 1468 non-null   object 
 6   crm_imja                        1468 non-null   object 
 7   crm_otchestvo                   1468 non-null   object 
 8   crm_familija                    1468 non-null   object 
 9   crm_dolzhnost                   1468 non-null   object 
 10  crm_nazvanie_kompanii           1468 non-null   object 
 11  crm_istochnik                   1468 non-null   object 
 12  crm_korporativnyj_sajt          14

**Основные метрики числовых данных**

In [37]:
df.describe()

,crm_id,crm_emkost_litsenzii,crm_summa,crm_uf_crm_input
count,1468.000000,1468.00000,1.468000e+03,0.0
mean,5089.611035,14.69346,1.144857e+04,NaN
std,3013.633512,23.90509,8.333752e+04,NaN
min,519.000000,0.00000,0.000000e+00,NaN
25%,2836.000000,10.00000,0.000000e+00,NaN
50%,4448.000000,10.00000,0.000000e+00,NaN
75%,8825.500000,10.00000,0.000000e+00,NaN
max,10349.000000,250.00000,1.512000e+06,NaN


**Удаление дубликатов**

In [38]:
# Удаление дубликатов по crm_id
df = df.drop_duplicates(subset='crm_id', keep="first")

In [39]:
df.head(5)

,crm_client_id,crm_id,crm_data_sozdanija,crm_status,crm_nazvanie_lida,crm_obraschenie,crm_imja,crm_otchestvo,crm_familija,crm_dolzhnost,...,crm_povtornyj_lid,crm_utm_source,crm_utm_medium,crm_utm_campaign,crm_utm_content,crm_utm_term,crm_klassifikator_otrasli,crm_otrasl,crm_otrasl_spisok,crm_uf_crm_input
0,1591642943190342147,10349,28.04.2021 11:21,Не обработан,Лид #10349,undefined,undefined,undefined,undefined,undefined,...,N,yandex,cpc,all_main_russia_poisk,sistema_upravlenie_sotrudnikami,undefined,undefined,"Управление недвижимостью, ЖКХ - Клининг",undefined,NaN
1,1618056694119409516,10347,28.04.2021 6:43,Обработан,ТОО TechQyzmet KZ,undefined,Алимжан Даржанов,undefined,undefined,Менеджер,...,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,NaN
2,1619554758624285553,10345,27.04.2021 23:24,Обработан,ИП Кирьянов,undefined,Евгений Кирьянов,undefined,undefined,ИП,...,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,NaN
3,1619526373198768982,10337,27.04.2021 15:31,Обработан,Сады Придонья,undefined,Елена Евгеньевна Слетова,undefined,undefined,undefined,...,N,undefined,undefined,undefined,undefined,undefined,"5. Промышленность, нефтегаз, ТЭК\t/\tОбслужива...","Промышленность, производство, ТЭК - Сервис обо...",undefined,NaN
4,undefined,10335,27.04.2021 11:44,Ожидает квалификации,Райт левел,Муж,Кирилл Герасимов,undefined,undefined,undefined,...,N,undefined,undefined,undefined,undefined,undefined,"7. Управление недвижимостью, ЖКХ\t/\tБЦ",Выездное обслуживание оборудования - Технологи...,undefined,NaN


In [40]:
df.shape

(1468, 32)

## Сохранение данных

In [41]:
# Сохранение в файл
df.to_csv('../data/td_crm.csv', index=False, encoding='utf-8', sep=';')